In [1]:
import sys
import argparse
import numpy as np
from scipy.stats import chi2_contingency
from itertools import product
import itertools as it

import allel
import pandas as pd

from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)
    
from datetime import datetime
import tempfile
import os
import gzip
import subprocess
import time

import collections
def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

In [2]:
from tools.mcounter_tools import (
    read_vcf_allel, ind_assignment_scatter_v1, MC_sample_matrix_v1,
    heatmap_v2, ind_assignment_SFS, read_windows_SFS
)

In [3]:
#from tools.SLiM_pipe_tools import mutation_counter_launch
import re
import pandas as pd


## directories
main_dir= os.getcwd() + '/'
sims_dir= main_dir + 'mutation_counter/data/sims_burnin/'
diffs= False
frequency_range= [0,1]
args= True

data_kmer, data= read_windows_SFS(diffs= diffs, frequency_range= frequency_range,indfile= 'ind_assignments.txt', outemp= 'ind_assignments{}.txt',
                    sim_dir= sims_dir, muted_dir= 'mutation_counter/data/mutation_count/',
                    outlog= 'indy.log', row= 24,col= 4, single= True, exclude= False,args= args)

missing: 0, no vcf: 0
400
AG
G T
3293 999999
1000000 3294
time elapsed: 591.7785725593567s


In [4]:
available= list(data.keys())
times_dict= {
    sim_t: int(data[sim_t]['args']['evt']) for sim_t in available
}

sorted_avail= sorted(times_dict,key= times_dict.get,reverse= False)

Nplot= 10
plot_whom= np.linspace(0,len(available)-1,Nplot,dtype=int)

n_bins= np.linspace(0,1.01,50)
pop= 'pop0'

fig= []

for sim_idx in plot_whom:
    sim_t= sorted_avail[sim_idx]
    gen_time= data[sim_t]['args']['evt']
    freqs= data[sim_t]['freqs'][pop]
    N= len(freqs)
    
    bin_middle= [(n_bins[x] + n_bins[x-1]) / 2 for x in range(1,len(n_bins))]

    bin_count= [[x for x in freqs if x >= n_bins[z-1] and x < n_bins[z]] for z in range(1,len(n_bins))]
    bin_count= [len(x) / N for x in bin_count]

    fig_sim= go.Scatter(
        x= bin_middle,
        y= bin_count,
        name= str(int(gen_time) + 1000)
    )
    
    fig.append(fig_sim)

layout= go.Layout(
    xaxis= dict(
        range= [-0.1,1.1],
        title= 'frequency'
    ),
    yaxis= dict(
        title= 'N'
    ),
    title= 'SFS by burnin'
)
Figure= go.Figure(data=fig,layout=layout)

iplot(Figure)

In [8]:
data['BurninT6411C8.52766618'].keys()

dict_keys(['freqs', 'inds', 'geno', 'summary'])

In [13]:
available= list(data.keys())
times_dict= {
    sim_t: int(data[sim_t]['args']['evt']) for sim_t in available
}

sorted_avail= sorted(times_dict,key= times_dict.get,reverse= False)

Nplot= 10
plot_whom= np.linspace(0,len(available)-1,Nplot,dtype=int)

Nbin= 50
n_bins= np.linspace(0,1.01,Nbin)
pop= 'pop0'

fig= go.Figure()

for sim_idx in plot_whom:
    sim_t= sorted_avail[sim_idx]
    gen_time= data[sim_t]['args']['evt']
    freqs= data[sim_t]['freqs'][pop]
    N= len(freqs)
    
    fig.add_trace(go.Histogram(
        x=freqs,
        histnorm='percent',
        name=str(int(gen_time) + 1000), # name used in legend and hover labels
        xbins=dict( # bins used for histogram
            start=0,
            end=1,
            size= 1 / 50
        ),
        #marker_color='#EB89B5',
        opacity=0.75
    ))
    

layout= go.Layout(
    xaxis= dict(
        range= [-0.1,1.1],
        title= 'frequency'
    ),
    yaxis= dict(
        title= 'N'
    ),
    title= 'SFS by burnin'
)

Figure['layout']= layout

#fig.to_image(format="png", width=2000, height=1500, scale=2)
iplot(Figure)

In [32]:

counts= []

for sim_idx in range(len(sorted_avail)):
    sim_t= sorted_avail[sim_idx]
    gen_time= data[sim_t]['args']['evt']
    #print(gen_time)
    freqs= data[sim_t]['freqs'][pop]
    N= len(freqs)

    n_bins= np.linspace(0,1.01,100)
    bin_middle= [(n_bins[x] + n_bins[x-1]) / 2 for x in range(1,len(n_bins))]
    
    bin_count= [[x for x in freqs if x >= n_bins[z-1] and x < n_bins[z]] for z in range(1,len(n_bins))]
    bin_count= [len(x) / N for x in bin_count]
    bin_count= np.array(bin_count)
    counts.append(bin_count)



In [33]:
burn_times= [times_dict[x] for x in sorted_avail]
burn_dict= {
    z: [x for x in range(len(burn_times)) if burn_times[x] == z] for z in list(set(burn_times))
}

burn_dict= {
    z: [counts[x] for x in burn_dict[z]] for z in burn_dict.keys()
}

burn_clades= sorted(list(burn_dict.keys()))

diffs= []
for idx in range(1,len(burn_clades)):
    
    ref= burn_clades[idx-1]
    burn= burn_clades[idx]
    
    dists= []
    
    
    for indian in burn_dict[burn]:
        
        dist_vec= [(x - indian) for x in burn_dict[ref]] #/ np.sum(indian + x)
        dist_vec= [z**2 for z in dist_vec]
        dist_vec= [np.sum(x) for x in dist_vec]
        dists.extend(dist_vec)
        
    diffs.append(dists)

    


In [34]:
mean_diffs= [np.mean(x) for x in diffs]
std_diffs= [np.std(x) for x in diffs]
surface= burn_clades[1:]



fig= [go.Scatter(
    x= surface,
    y= mean_diffs,
    error_y= dict(
        array= std_diffs,
        type= 'data',
        #symmetric= True,
        visible=True
    )
)]

layout= go.Layout(
    title= 'SFS distances by burnin time',
    xaxis= dict(
        title= 'time before sampling',
        range= [0, max(burn_times) + 1000]
    ),
    yaxis= dict(
        title= 'sum of squared diffs'
    )
)
Figure= go.Figure(data= fig,layout=layout)

iplot(Figure)